Math shared by all implementations

All implementations have the fundamental bits:

input matrix X of size [num_samples, num_features]

output matrix y of size [num_samples, num_outputs]

projection matrix W of size [num_features, num_neurons]

non-linear tranformation function ufunc like hyperbolic tangent (np.tanh), sigmoid, or any other

output weights B of size [num_neurons, num_outputs]

optional output bias/intercept of size [num_outputs, ]


In [600]:
import numpy as np
import pandas  as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from scipy import linalg
from math import sqrt

In [601]:
class clientClass:
   
    def __init__(self,client_id,batch_size,h_neurons=20, alpha=0.001):
        
        self.i=client_id
        cl= np.load(f"client{self.i}.npz")

        if client_id == 1:
            n = 100
        elif client_id == 2:
            n = 1000
        else:
            n = 10000
            
        
        X_t = cl[f"X_a{self.i}_sc"][:n]
        X_s = cl[f"X_a{self.i}_sc"][n:]
        y_t = cl[f"y_a{self.i}"][:n]
        y_s = cl[f"y_a{self.i}"][n:]

           
        self.input_size = len(X_t)
        self.n_features = len(X_t[1])
        self.h_neurons=h_neurons
        self.w = None
        self.b = None
        self.batch_size=batch_size
        self.reg_term = np.eye(self.h_neurons) * alpha
        self.X_t = X_t
        self.X_s = X_s
        self.y_t = y_t
        self.y_s = y_s
          
            
    def get_Nfs(self):
        return self.input_size, self.n_features
    
    def set_W_b(self,w,b):
        self.w=w
        self.b=b  
    
    def batches(self):
        
        batch_X=[]
        batch_y=[]
        
        if self.batch_size >= self.input_size:
            batch_X.append(self.X_t)
            batch_y.append(self.y_t)
             
        else:
            for i in range (0, self.input_size , self.batch_size):
                batch_X.append(self.X_t[i:i + self.batch_size-1])
                batch_y.append(self.y_t[i:i + self.batch_size-1])
            
        for X,y in zip(batch_X,batch_y): 
            
             # Compute the hidden layer output 
            H = np.tanh(np.dot(X, self.w) + self.b)  # Apply activation function (tanh)

            HTH = H.T @ H + self.reg_term
            HTy = H.T @ y
              
            yield HTH, HTy               
   
    def predict_test(self, B):
                                    
        # Calculate the predicted output on the testing set
        H_test = np.tanh(np.dot(self.X_s, self.w) + self.b)
        pred_test = np.dot(H_test, B)
        r2 = r2_score(self.y_s, pred_test)
        return r2


In [763]:


def par_B(cl1,cl2,cl3):
    # Perform some operation using a for loop
    # combine some client data
    clients = [cl1, cl2, cl3]
    HtH1 = None
    HtY1 = None
    Bd1={}
    Bd=[]

    for client in clients:
         
        data = client.batches()

        for hth, hty in data:

            HtH1 = hth
            HtY1 = hty

            #print(f"Client   : {client.get_Nfs()}")
            Bd.append(np.linalg.solve(HtH1, HtY1))
            #print(f"Bd: {Bd}")
        Bd1[client.i] = Bd 
        Bd=[]

    return Bd1

In [764]:
# I am a server now
# initialize ELM parameters shared among clients

hidden_size=20

# initialize clients, server sends ELM parameters to clients
cl1 = clientClass(1, batch_size=100, h_neurons=hidden_size, alpha=0.001)
cl2 = clientClass(2, batch_size=300, h_neurons=hidden_size, alpha=0.001)
cl3 = clientClass(3, batch_size=1000, h_neurons=hidden_size, alpha=0.001)

#cl1_input_size,cl1_n_features =cl1.get_Nfs()
#cl2_input_size,cl2_n_features =cl2.get_Nfs()
#cl3_input_size,cl3_n_features =cl3.get_Nfs()

input_size=8
w_inital = np.random.rand(input_size, hidden_size)
bias_inital = np.random.rand(hidden_size)

#clients = [cl1, cl2, cl3]
cl1.set_W_b(w_inital, bias_inital)
cl2.set_W_b(w_inital, bias_inital)
cl3.set_W_b(w_inital, bias_inital)


In [765]:
def average_matrices_Sl(matrix1):
    # Check if the matrices have the same shape
    if len(matrix1) == 0:
        return None  # Handle the case of an empty list
    average_matrix = sum(matrix1 ) / len(matrix1)
    return average_matrix
    

In [766]:
def global_B(B):
    ave_client123=[]
    for i in range(1,len(B)+1):
        print(f"Client_{i}")
        ave_client123.append(average_matrices_Sl(B[i]))
        print(f"{average_matrices_Sl(B_1i[i])}")
        print()
    print(f"global_B: {average_matrices_Sl(ave_client123)}")
    return average_matrices_Sl(ave_client123)



In [767]:
B_1i=par_B(cl1,cl2,cl3)

In [768]:
B_1i

{1: [array([-0.16933469, -1.000057  ,  0.4128051 ,  2.10781281, -0.95159671,
         -1.02649389,  0.26680507, -0.05615385, -0.40754701, -0.06856119,
          1.87422439,  1.49143716, -0.20454003,  0.5042078 , -1.538691  ,
         -0.46490047, -0.97797467, -0.67908325,  0.02680227,  1.15890938])],
 2: [array([ -0.31442912,  -0.58895931,  -0.21350481,   1.61276705,
          -2.4415261 ,   0.74929013,   0.70311663,   0.50170554,
          -1.19452076,  13.10196772,  -0.33561967,   0.40369911,
           0.27954207,  -0.39840468,   2.04155327,  -0.41355005,
         -13.71367854,  -0.22094864,  -0.77103194,  -0.01463847]),
  array([ 0.4180349 , -0.70450403, -0.14435629, -3.10292407,  0.63002801,
          0.87887043, -0.85578475,  0.62496659,  1.32339078, -0.75028125,
         -0.31070456,  0.58520712,  0.03453499, -0.19541292,  1.97320361,
         -0.41400257,  0.66200978, -0.52420127, -1.74027955,  0.04074105]),
  array([-0.09881171, -0.75533648, -0.57873447, -0.2287038 ,  3.406290

In [769]:
g_B_it1=global_B(B_1i)

Client_1
[-0.16933469 -1.000057    0.4128051   2.10781281 -0.95159671 -1.02649389
  0.26680507 -0.05615385 -0.40754701 -0.06856119  1.87422439  1.49143716
 -0.20454003  0.5042078  -1.538691   -0.46490047 -0.97797467 -0.67908325
  0.02680227  1.15890938]

Client_2
[-0.33628677 -0.69969953 -0.12136388 -0.35747627  0.19205442  0.72137046
  0.17593658  0.31294033 -0.79802498  2.6925046  -0.15804976  0.52291058
  0.12145033 -0.15785553  1.44549926 -0.16131542 -3.98485974 -0.50618713
 -0.73947996  0.19316326]

Client_3
[-0.68309589 -1.28145506 -0.3847183   0.61855577 -0.26029972  1.0813024
  1.02028338  0.26714612 -0.79564531 -1.74438929 -0.0388207   0.51356585
  0.08191829  0.1191554   0.30575013 -0.28035536 -0.06922205 -0.30920552
  0.07177608  0.06024854]

global_B: [-3.96239118e-01 -9.93737197e-01 -3.10923578e-02  7.89630768e-01
 -3.39947337e-01  2.58726323e-01  4.87675010e-01  1.74644202e-01
 -6.67072435e-01  2.93184704e-01  5.59117977e-01  8.42637863e-01
 -3.90469550e-04  1.55169223e-0

In [770]:
g_B_it1

array([-3.96239118e-01, -9.93737197e-01, -3.10923578e-02,  7.89630768e-01,
       -3.39947337e-01,  2.58726323e-01,  4.87675010e-01,  1.74644202e-01,
       -6.67072435e-01,  2.93184704e-01,  5.59117977e-01,  8.42637863e-01,
       -3.90469550e-04,  1.55169223e-01,  7.08527956e-02, -3.02190420e-01,
       -1.67735215e+00, -4.98158633e-01, -2.13633872e-01,  4.70773730e-01])

In [771]:
r2_it1=cl1.predict_test(g_B_it1)
r2_it1

-25.117573224198665

In [772]:
r2_it2=cl2.predict_test(g_B_it1)
r2_it2

-0.42329902970394784

In [773]:
r2_it3=cl3.predict_test(g_B_it1)
r2_it3

-0.41988482209820366